In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
from collections import Counter
import matplotlib.pyplot as plt

In [5]:
df = pd.read_excel("../dataset/data_train_indobert.xlsx")

DATA INTEGRATION

In [ ]:
df["text"] = (df["Nama Produk"] + " " + df["Deskripsi Produk"])

df = df.drop(columns=['Nama Produk', 'Deskripsi Produk'])

kolom_baru = ['text'] + [col for col in df.columns if col != 'text']
df = df[kolom_baru]

# df.to_csv("../dataset/data_integration_train.csv", index=False, encoding='utf-8-sig', sep=';')
# print("File data_integration_train.csv berhasil disimpan")

print("\nPreview data:")
df[["text"]].head()

File data_integration_train.csv berhasil disimpan

Preview data:


,text
0,SANMOL FORTE 650 MG STRIP 4 TABLET Deskripsi:\...
1,SANMOL 500 mg / SANMOL TABLET (HARGA/ STRIP) T...
2,SANMOL 500 MG STRIP 4 TABLET Deskripsi:\nSanmo...
3,ERPHAMOL FORTE 650 mg BOX Paracetamol-Obat Dem...
4,PARACETAMOL TRIMAN 500 MG STRIP 10 KAPLET Desk...


DATA CLEANING

In [7]:
def clean_text(text):
    text = str(text)
    text = unicodedata.normalize("NFKD", text)
    text = re.sub(r"[^\x00-\x7F]", " ", text)
    text = re.sub(r"[=]{2,}", " ", text)
    text = re.sub(r"[!#*~]{2,}", " ", text)
    text = re.sub(r"\n{2,}", "\n", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

In [9]:
print("Memulai pembersihan... Menampilkan maksimal 5 baris pertama yang berubah:\n")

preview_count = 0
preview_limit = 5 

for index, row in df.iterrows():
    text_original = row['text']
    text_cleaned = clean_text(text_original)

    df.loc[index, 'text'] = text_cleaned

    if text_original != text_cleaned and preview_count < preview_limit:
        removed_chars = set(text_original) - set(text_cleaned)
        chars_to_show = [char for char in removed_chars if not char.isspace()]
        
        if chars_to_show:
            sample_chars = chars_to_show[:3]
            print(f"Baris ke-{index}: Dihapus karakter seperti -> {', '.join(sample_chars)}")
        else:
            print(f"Baris ke-{index}: Dihapus spasi/baris baru berlebih")
            
        preview_count += 1

print("\n✅ Proses pembersihan seluruh data selesai.")
print("\nDataFrame setelah dibersihkan:")
print(df)

Memulai pembersihan... Menampilkan maksimal 5 baris pertama yang berubah:


✅ Proses pembersihan seluruh data selesai.

DataFrame setelah dibersihkan:
                                                   text
0     SANMOL FORTE 650 MG STRIP 4 TABLET Deskripsi:\...
1     SANMOL 500 mg / SANMOL TABLET (HARGA/ STRIP) T...
2     SANMOL 500 MG STRIP 4 TABLET Deskripsi:\nSanmo...
3     ERPHAMOL FORTE 650 mg BOX Paracetamol-Obat Dem...
4     PARACETAMOL TRIMAN 500 MG STRIP 10 KAPLET Desk...
...                                                 ...
8500  Promo Terbatas - Madu Hijau Menjaga Kesehatan ...
8501  Madu Asmafas / Asma Dan Sesak Nafas / Napas Ma...
8502  Diskon Ling Shen Yao Serbuk Obat Herbal Kanker...
8503  TERLARIS " HERBAL ASAM URAT SUPER" SELAMAT DAT...
8504  Kunir Putih Windra Mekar Curcuval DP (60 kapsu...

[8505 rows x 1 columns]


In [10]:
# Hapus duplikat
before = len(df)
df = df.drop_duplicates(subset=["text"])
after = len(df)

print(f"Jumlah data sebelum: {before}, sesudah: {after}")

Jumlah data sebelum: 8505, sesudah: 4582


In [11]:
df.dropna(subset=['text'], inplace=True)
df = df[df['text'].str.lower() != 'nan']

# df.to_csv("data_cleaned_train.csv", index=False, encoding='utf-8-sig', sep=';')
# print("File data_cleaned_train.csv berhasil disimpan")
print(f"total data: {len(df)}")

total data: 4581


In [12]:
df.to_csv("../dataset/preprocessed_indobert_data_train.csv", index=False, sep=';')